In [1]:
import pandas as pd
import urllib
from lxml import etree, html

In [2]:
def get_submission_trtables(weburl):
    with urllib.request.urlopen(weburl) as response:
       page = response.read()
    doc = html.fromstring(page)
    table = doc.xpath("//*[contains(@class, 'border_white')]")[0]
    tr_elements = table.xpath('tr')
    return tr_elements

In [3]:
def get_array_from_trtable(tr_elements, filename=None):
    col=[]
    i=0

    # Make header
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        col.append((name,[]))

    for j in range(1,len(tr_elements)):
        # T is our j'th row
        T=tr_elements[j]

        # index of the column
        i=0

        # Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            if i==0:
                data= str(data).strip()
            col[i][1].append(data)
            i+=1
            
    Dict = {title:column for (title,column) in col}
    submissions = pd.DataFrame(Dict)
    if filename is not None:
        submissions.to_csv(filename + '.csv', sep='\t')
    return submissions

In [4]:
weburl = 'http://www.tractometer.org/ismrm_2015_challenge/results'
with urllib.request.urlopen(weburl) as response:
   page = response.read()
doc = html.fromstring(page)
table = doc.xpath("//*[contains(@id, 'results_table')]")[0]
tr_elements = table.xpath('//tr')
submissions = get_array_from_trtable(tr_elements, filename='scraped_submissions')

In [5]:
submission_url = 'http://www.tractometer.org/ismrm_2015_challenge/results/{0}'
submission_tables = {}
for submission in submissions['Submission ID']:
    trelements = get_submission_trtables(submission_url.format(submission))
    submission_tables[submission] = get_array_from_trtable(trelements, filename='submission{0}'.format(submission))
    

In [6]:
submissions

,Submission ID,Team ID,File ID,VB / 25,IB,VC (%),VCWP (%),IC (%),NC (%),Mean crossing angularerror (degrees)
0,1_0,1,0,23,79,62.18,0.00,27.78,10.04,27.95
1,1_1,1,1,23,77,63.87,0.00,23.77,12.36,23.34
2,1_2,1,2,23,82,62.98,0.00,24.06,12.96,23.19
3,1_3,1,3,23,71,63.72,0.00,25.34,10.93,23.94
4,1_4,1,4,23,93,62.47,0.00,26.96,10.57,30.35
...,...,...,...,...,...,...,...,...,...,...
91,9_0,9,0,24,127,51.75,0.00,41.00,7.25,26.25
92,9_1,9,1,23,185,29.86,0.00,64.85,5.29,28.84
93,9_2,9,2,23,162,31.47,0.00,62.80,5.73,27.75
94,9_3,9,3,23,141,49.69,0.00,42.61,7.70,27.40


In [7]:
submission_tables['1_0']

,Bundle,Count,% of totalstreamlines,Overlap (% of GT),Overreach (% of GT)
0,Superior Cerebellar Peduncle (right),282,0.282%,43.618%,31.924%
1,Parieto-Occipito Pontine Tract (left),875,0.875%,39.599%,25.183%
2,Cingulum (right),4171,4.171%,50.380%,25.220%
3,Corpus Callosum,20849,20.849%,61.401%,26.852%
4,Anterior Commissure,0,0.000%,0.000%,0.000%
5,Fronto-Pontine Tract (right),1530,1.530%,30.643%,19.251%
6,Posterior Commissure,0,0.000%,0.000%,0.000%
7,Inferior Cerebellar Peduncle (left),1378,1.378%,71.550%,61.784%
8,Fronto-Pontine Tract (left),641,0.641%,21.359%,13.671%
9,Uncinate Fasciculus (right),1963,1.963%,69.495%,46.199%


In [9]:
submissions['mean overlap'] = -1
submissions['mean overreach'] = -1
for team, submission in submission_tables.items():
    ol_mean = submission['Overlap (% of GT)'].str.replace("%", "").astype(float).mean()
    or_mean = submission['Overreach (% of GT)'].str.replace("%", "").astype(float).mean()
    
    idx = submissions.index[submissions['Submission ID'] == team]
    submissions.loc[idx, 'mean overlap'] = ol_mean
    submissions.loc[idx, 'mean overreach'] = or_mean

In [10]:
submissions

,Submission ID,Team ID,File ID,VB / 25,IB,VC (%),VCWP (%),IC (%),NC (%),Mean crossing angularerror (degrees),mean overlap,mean overreach
0,1_0,1,0,23,79,62.18,0.00,27.78,10.04,27.95,47.08648,31.19656
1,1_1,1,1,23,77,63.87,0.00,23.77,12.36,23.34,48.47968,31.43680
2,1_2,1,2,23,82,62.98,0.00,24.06,12.96,23.19,48.80416,32.03640
3,1_3,1,3,23,71,63.72,0.00,25.34,10.93,23.94,47.78560,35.26424
4,1_4,1,4,23,93,62.47,0.00,26.96,10.57,30.35,35.24576,9.57652
...,...,...,...,...,...,...,...,...,...,...,...,...
91,9_0,9,0,24,127,51.75,0.00,41.00,7.25,26.25,35.44028,32.36844
92,9_1,9,1,23,185,29.86,0.00,64.85,5.29,28.84,64.66564,103.93892
93,9_2,9,2,23,162,31.47,0.00,62.80,5.73,27.75,47.67232,58.49068
94,9_3,9,3,23,141,49.69,0.00,42.61,7.70,27.40,41.32188,45.49960
